<a href="https://colab.research.google.com/github/fmarquezf/MetNumUN2021I/blob/main/Lab11/AlgorithmicToolboxWeek2QRdecompositionGroup4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [ ]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    # ... ENTER YOUR CODE HERE ...
    x = vec
    y = np.zeros((len(x)))
    y[0] = np.linalg.norm(x)

    u = (x - y) / np.linalg.norm(x - y)

    H = np.eye(len(x)) - 2*np.outer(u,u)
    outvec = H@x
    return outvec, H

Test your function using tests below:

In [ ]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [ ]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [ ]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> 
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    q = np.eye(m)
    r = a1.copy()

    # ... ENTER YOUR CODE HERE ...
    for i in range(n):
      v1, h = householder(r[i:,i])
      H = np.eye(m)
      H[i:,i:] = h
      r = H@r
      q = q@H
    return q, r

In [ ]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [ ]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [ ]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [ ]:
q

array([[ 0.13665049,  0.53601299, -0.09369752,  0.7697136 ,  0.30459557],
       [ 0.56035895,  0.0935397 , -0.53326881,  0.01839528, -0.62652547],
       [ 0.19725922,  0.65948912,  0.60068463, -0.32384673, -0.24589462],
       [ 0.62498418, -0.50418303,  0.52144688,  0.28453698,  0.04822969],
       [ 0.48765568,  0.12171264, -0.27224305, -0.47049398,  0.67223293]])

In [ ]:
qq

array([[-0.13665049,  0.53601299,  0.09369752,  0.661619  , -0.49749149],
       [-0.56035895,  0.0935397 ,  0.53326881, -0.52477245, -0.34276292],
       [-0.19725922,  0.65948912, -0.60068463, -0.37879015,  0.14784752],
       [-0.62498418, -0.50418303, -0.52144688,  0.18967657, -0.21750907],
       [-0.48765568,  0.12171264,  0.27224305,  0.32774225,  0.75222783]])

In [ ]:
r

array([[ 1.40152769,  1.2514379 ,  0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [-0.        , -0.        ,  0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ]])

In [ ]:
rr

array([[-1.40152769, -1.2514379 , -0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        ,  0.        , -0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

*Enter your explanation here* (10% of the total grade, peer-graded)
The matrices are pretty similar, except for the fact that the signs are not always correct because of the matrix rotation, and the last 2 columns of Q are different to the ones from qq, thats because the size of R makes it last 2 rows to be 0, and that doesn't let calculate the last 2 columns of Q correctly.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

**1) QR for Gram Schmidt Method**

In [ ]:
def gram_schmidt(A):
    """Gram-schmidt orthogonalization"""
    Q=np.zeros_like(A)
    cnt = 0
    for a in A.T:
        u = np.copy(a)
        for i in range(0, cnt):
            u -= np.dot(np.dot(Q[:, i].T, a), Q[:, i]) # Subtract the projection of the vector to be sought on the vector
        e = u / np.linalg.norm(u)  # Normalized
        Q[:, cnt] = e
        cnt += 1
    R = np.dot(Q.T, A)
    return (Q, R)

In [ ]:
#test Q@R = qq@rr
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
Q_gs, R_gs = gram_schmidt(a)
qq_gs, rr_gs = qr(a)

assert_allclose(np.dot(Q, R), np.dot(qq, rr))

Checking if q and r agree with qq and rr

In [ ]:
Q_gs

array([[ 0.13665049,  0.53601299, -0.09369752],
       [ 0.56035895,  0.0935397 , -0.53326881],
       [ 0.19725922,  0.65948912,  0.60068463],
       [ 0.62498418, -0.50418303,  0.52144688],
       [ 0.48765568,  0.12171264, -0.27224305]])

In [ ]:
qq_gs

array([[-0.13665049,  0.53601299,  0.09369752,  0.661619  , -0.49749149],
       [-0.56035895,  0.0935397 ,  0.53326881, -0.52477245, -0.34276292],
       [-0.19725922,  0.65948912, -0.60068463, -0.37879015,  0.14784752],
       [-0.62498418, -0.50418303, -0.52144688,  0.18967657, -0.21750907],
       [-0.48765568,  0.12171264,  0.27224305,  0.32774225,  0.75222783]])

In [ ]:
R_gs

array([[ 1.40152769,  1.2514379 ,  0.89523615],
       [-0.        ,  0.84158252,  0.68447942],
       [ 0.        ,  0.        ,  0.5496046 ]])

In [ ]:
rr_gs

array([[-1.40152769, -1.2514379 , -0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        ,  0.        , -0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

**2) QR for Givens Rotation Method**

In [ ]:
def givens_rotation(A):
    """Givens Transformation"""
    (r, c) = np.shape(A)
    Q = np.identity(r)
    R = np.copy(A)
    (rows, cols) = np.tril_indices(r, -1, c)
    for (row, col) in zip(rows, cols):
        if R[row, col] != 0:  # R[row, col]=0 then c=1, s=0, R and Q remain unchanged
            r_ = np.hypot(R[col, col], R[row, col])  # d
            c = R[col, col]/r_
            s = -R[row, col]/r_
            G = np.identity(r)
            G[[col, row], [col, row]] = c
            G[row, col] = s
            G[col, row] = -s
            R = np.dot(G, R)  # R=G(n-1,n)*...*G(2n)*...*G(23,1n)*...*G(12)*A
            Q = np.dot(Q, G.T)  # Q=G(n-1,n).T*...*G(2n).T*...*G(23,1n).T*...*G(12).T
    return (Q, R)

In [ ]:
#test Q@R = qq@rr
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
Q_gr, R_gr = givens_rotation(a)
qq_gr, rr_gr = qr(a)

assert_allclose(np.dot(Q, R), np.dot(qq, rr))

Checking if q and r agree with qq and rr

In [ ]:
Q_gr

array([[ 0.13665049,  0.53601299, -0.09369752, -0.80526127, -0.19181182],
       [ 0.56035895,  0.0935397 , -0.53326881,  0.34418264, -0.5238424 ],
       [ 0.19725922,  0.65948912,  0.60068463,  0.4063158 , -0.01575883],
       [ 0.62498418, -0.50418303,  0.52144688, -0.26076822, -0.12364196],
       [ 0.48765568,  0.12171264, -0.27224305,  0.        ,  0.82052525]])

In [ ]:
qq_gr

array([[-0.13665049,  0.53601299,  0.09369752,  0.661619  , -0.49749149],
       [-0.56035895,  0.0935397 ,  0.53326881, -0.52477245, -0.34276292],
       [-0.19725922,  0.65948912, -0.60068463, -0.37879015,  0.14784752],
       [-0.62498418, -0.50418303, -0.52144688,  0.18967657, -0.21750907],
       [-0.48765568,  0.12171264,  0.27224305,  0.32774225,  0.75222783]])

In [ ]:
R_gr

array([[ 1.40152769,  1.2514379 ,  0.89523615],
       [-0.        ,  0.84158252,  0.68447942],
       [ 0.        , -0.        ,  0.5496046 ],
       [-0.        ,  0.        , -0.        ],
       [ 0.        , -0.        , -0.        ]])

In [ ]:
rr_gr

array([[-1.40152769, -1.2514379 , -0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        ,  0.        , -0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])